In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from geopandas import GeoDataFrame
from shapely.geometry import Point
from geopy.geocoders import ArcGIS
import os

In [2]:
path = r'T:\MPO\TIP\TIP FY24-27\Maps\Data'

In [3]:
df = pd.read_excel(os.path.join(path, 'AddCoordinates.xlsx'))

In [4]:
df

,Project Name,STIP Key,Longitude,Latitude
0,"Glenwood Transit Facilities Treatments, LTD (5...",23208,-123.041142,44.042793
1,LTD EmX Franklin & Gateway Station Treatments ...,23220,-123.042728,44.073423
2,LTD RideSource Facility Expansion (5307),23223,-123.123155,44.057281
3,LTD Glenwood Board Room Upgrades (5307),23230,-123.041362,44.042346
4,Lane County Local Road Curve Treatments (2027),22689,-123.210154,44.102908
5,Lane County Local Road Curve Treatments (2027),22689,-123.215336,44.040951


In [5]:
df['ID'] = list(range(32, 32 + df.shape[0]))

In [6]:
geometry = [Point(xy) for xy in zip(df.Longitude, df.Latitude)]
gdf = GeoDataFrame(df, crs="EPSG:4326", geometry=geometry)

In [7]:
points = gpd.read_file(os.path.join(path, 'FY24_27_points.shp'))

In [8]:
gdf.rename(columns={'STIP Key':'STIP_Key'}, inplace=True)

In [11]:
gdf = gdf.to_crs(epsg=2992)

In [14]:
points_added = points.append(gdf[points.columns], ignore_index=True)

In [31]:
locs = ["Pioneer Pkwy E & E St, Springfield, OR 97477", 
        "E St & Pioneer Pkwy W, Springfield, OR 97477", 
        "69th St & Thurston Rd, Springfield, OR 97478"]

In [71]:
def getCoordinates(address):
    lonlat = ArcGIS().geocode(address)
    #latitude = lonlat[1][0]
    #longitude = lonlat[1][1]
    (latitude, longitude) = lonlat[1]
    return [address, longitude, latitude] 

In [75]:
# Create the pandas DataFrame
ndf = pd.DataFrame(dt, columns = ['Address', 'Longitude', 'Latitude'])

In [76]:
ndf['STIP_Key'] = 22351
ndf['ID'] = 16

In [78]:
geometry = [Point(xy) for xy in zip(ndf.Longitude, ndf.Latitude)]
ngdf = GeoDataFrame(ndf, crs="EPSG:4326", geometry=geometry)

In [79]:
ngdf = ngdf.to_crs(epsg=2992)

In [82]:
npoints_added = points_added.append(ngdf[points.columns], ignore_index=True)

In [84]:
data = pd.read_excel(os.path.join(path, 'MTIP24_27_List_Updated.xlsx'))

In [85]:
lines = gpd.read_file(os.path.join(path, 'FY24_27_lines.shp'))

In [96]:
Ids = npoints_added[['STIP_Key', 'ID']].drop_duplicates().append(lines[['STIP_Key', 'ID']].drop_duplicates(), ignore_index=True)

In [97]:
Ids.rename(columns={'STIP_Key':'STIP Key'}, inplace=True)

In [98]:
Ids['STIP Key'] = Ids['STIP Key'].astype(np.int64)

In [89]:
data.shape

(208, 27)

In [100]:
ndata = data.merge(Ids, how='outer', on='STIP Key')

In [101]:
ndata.shape

(236, 28)

In [102]:
ndata.to_excel(os.path.join(path, 'MTIP24_27_List_IDs.xlsx'), 
               sheet_name='2024-2027 Project List', 
               index=False)  

In [94]:
# overwrite the existing data
npoints_added.to_file(os.path.join(path, 'FY24_27_points.shp'))